# Writing and reading data

In [1]:
import fiona
from fiona.collection import Collection, supported_drivers

In [2]:
supported_drivers

{'ARCGEN': 'r',
 'AeronavFAA': 'r',
 'BNA': 'raw',
 'DGN': 'raw',
 'DXF': 'raw',
 'ESRI Shapefile': 'raw',
 'FileGDB': 'raw',
 'GMT': 'raw',
 'GPKG': 'rw',
 'GPSTrackMaker': 'raw',
 'GPX': 'raw',
 'GeoJSON': 'rw',
 'Idrisi': 'r',
 'MapInfo File': 'raw',
 'OpenFileGDB': 'r',
 'PCIDSK': 'r',
 'PDS': 'r',
 'SEGY': 'r',
 'SUA': 'r'}

## Writing a shapefile from a CSV

- reading a CSV from CartoDB SQL API (quering [this table](https://team.cartodb.com/u/jsanz/tables/twitter_pycones_pycones2015_pycones15/public) using requests)
- creating a schema
- creating features and writing them

In [3]:
import requests

In [4]:
url = 'https://jsanz.cartodb.com:443/api/v2/sql?q=select ST_X(the_geom) as lon, ST_Y(the_geom) as lat, cartodb_id, actor_preferredusername, body, postedtime from jsanz.twitter_pycones_pycones2015_pycones15&format=csv'
csv_file = '/tmp/tweets.csv'

req = requests.get(url)

with open(csv_file,'w') as csvfile:
    csvfile.write(req.text.encode('utf8','replace'))

In [5]:
from fiona.crs import from_epsg

In [6]:
target = '/tmp/tweets.shp'
epsg = 4258

schema = {
        "geometry": "Point",
        "properties": {
            ("cartodb_id", "int"),
            ("lon","float"),
            ("lat","float"),
            ("author","str"),
            ("body","str"),
            ("postedtime","str")
        }
}


In [8]:
import csv

In [9]:
#['lon,lat,cartodb_id,actor_preferredusername,body,postedtime',
# '-3.69063,40.42526,187,rmajadas,@PyConES listo!,2015-11-01 08:46:04+00']
driver = "ESRI Shapefile"
crs = from_epsg(epsg)
output = fiona.open(target, "w", driver=driver, crs=crs, schema=schema)
with open(csv_file,'r') as csvfile:
    csvreader = csv.reader(csvfile,delimiter=',',quotechar='"')
    next(csvreader)
    for line in csvreader:
            x = float(line[0])
            y = float(line[1])
            
            feature = {
                    "geometry" : {
                            "coordinates" : (x, y),
                            "type" : "Point"
                    },
                    "properties" : {
                            "cartodb_id" : int(line[2]),
                            "lon" : x,
                            "lat" : y,
                            "author" : line[3],
                            "body" : line[4].decode('latin1','replace'),
                            "postedtime" : line[5]
                    }
            }
            output.write(feature)

    output.close()

## Reading and exploring data

- Reading a file
- Getting information of the resource (bbox, CRS)?
- Looping ofver features, features API

In [10]:
source = fiona.open(target, 'r')

In [11]:
print "bounds: {}".format(source.bounds)

bounds: (-95.36327, -43.24895, 121.0, 53.33306)


In [12]:
print "CRS: {}".format(source.crs)

CRS: {u'no_defs': True, u'ellps': u'GRS80', u'proj': u'longlat'}


In [13]:
for f in source[:10]:
    print "{:3} - {}".format(f['properties']['cartodb_id'], f['geometry']['coordinates'])

187 - (-3.69063, 40.42526)
 39 - (2.15899, 41.38879)
249 - (-3.60667, 37.18817)
247 - (-16.25462, 28.46824)
105 - (-0.12574, 51.50853)
206 - (-3.69063, 40.42526)
192 - (-3.69063, 40.42526)
  9 - (-4.52406, 42.00955)
 35 - (2.15899, 41.38879)
 99 - (-0.12574, 51.50853)
